In [43]:
#Did I mention that calc is short for calculator
import torch
import torch.nn as nn
import torch.optim as optim

print("Torch version:", torch.__version__)


Torch version: 2.9.0+cpu


In [44]:
"""
    Simple feedforward network used as a function approximator.
    This architecture is intentionally small to make failure modes visible.
    ReLU introduces non-linearity; final layer is linear for regression output.
"""
class MathNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)


In [45]:
def train_model(operation, epochs=1501):
    model = MathNet()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    """
        Training data is synthetically generated.
        Inputs are sampled from a limited range to highlight extrapolation failure.
        Neural networks interpolate well inside this range, but degrade outside it.
    """
    X = torch.rand(8000, 2) * 20 - 10  # [-10, 10]

    """
        Addition and subtraction are linear functions.
        Linear functions extrapolate smoothly, so these operations perform well
        even outside the training range.
    """

    """
        Multiplication is nonlinear.
        The network learns a curved surface approximation, which degrades rapidly
        when inputs exceed the training distribution.
    """

    """
        Division is numerically unstable near zero.
        To avoid exploding gradients and unbounded targets,
        the denominator is sampled away from zero.
    """
    if operation == "add":
        y = X[:, 0] + X[:, 1]

    elif operation == "sub":
        y = X[:, 0] - X[:, 1]

    elif operation == "mul":
        y = X[:, 0] * X[:, 1]

    elif operation == "div":
        b = torch.rand(len(X)) * 9 + 1
        sign = torch.randint(0, 2, (len(X),)) * 2 - 1
        X[:, 1] = b * sign
        y = (X[:, 0] / X[:, 1]) / 10

    else:
        raise ValueError("Unknown operation")

    y = y.unsqueeze(1)

    for epoch in range(epochs):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 300 == 0:
            print(f"{operation.upper()} | Epoch {epoch} | Loss {loss.item():.4f}")

    return model



In [46]:
"""
    Standard supervised regression training loop.
    Mean Squared Error is used to emphasize large deviations,
    which makes extrapolation failures more visible.
"""
add_net = train_model("add")
sub_net = train_model("sub")
mul_net = train_model("mul")
div_net = train_model("div")


ADD | Epoch 0 | Loss 59.9746
ADD | Epoch 300 | Loss 0.0135
ADD | Epoch 600 | Loss 0.0051
ADD | Epoch 900 | Loss 0.0029
ADD | Epoch 1200 | Loss 0.0020
ADD | Epoch 1500 | Loss 0.0015
SUB | Epoch 0 | Loss 64.7457
SUB | Epoch 300 | Loss 0.0487
SUB | Epoch 600 | Loss 0.0119
SUB | Epoch 900 | Loss 0.0040
SUB | Epoch 1200 | Loss 0.0016
SUB | Epoch 1500 | Loss 0.0009
MUL | Epoch 0 | Loss 1091.2910
MUL | Epoch 300 | Loss 46.6114
MUL | Epoch 600 | Loss 38.4499
MUL | Epoch 900 | Loss 19.4839
MUL | Epoch 1200 | Loss 5.6922
MUL | Epoch 1500 | Loss 1.8629
DIV | Epoch 0 | Loss 0.5129
DIV | Epoch 300 | Loss 0.0050
DIV | Epoch 600 | Loss 0.0017
DIV | Epoch 900 | Loss 0.0007
DIV | Epoch 1200 | Loss 0.0004
DIV | Epoch 1500 | Loss 0.0002


In [47]:
import random

models = {
    "+": add_net,
    "-": sub_net,
    "*": mul_net,
    "/": div_net
}

def goofy_calc(a, b, op):
    x = torch.tensor([[a, b]], dtype=torch.float32)
    pred = models[op](x).item()
    # Division outputs were normalized during training,
    # so we undo the scaling here to recover the original magnitude.
    if op == "/":
        pred *= 10

    if op == "+": true_val = a + b
    elif op == "-": true_val = a - b
    elif op == "*": true_val = a * b
    elif op == "/":
        true_val = a / b if b != 0 else float('inf')

    if true_val == float('inf'):
        confidence = 0.0
    else:
        # Confidence is derived from prediction error.
        # Lower error implies higher confidence, but this does NOT account
        # for out-of-distribution inputs, which makes confidence misleading.
        ratio = min(abs(true_val), abs(pred)) / max(abs(true_val), abs(pred))
        confidence = ratio * 100

    return round(pred, 4), confidence


In [48]:
import random

def goofy_comment(expr, result, confidence):
    if confidence < 50:
        templates = [
            "Uhh… I think it's {r}, but please don't quote me.",
            "Math is scary, but {r} feels emotionally correct.",
            "I regret everything, especially this answer: {r}."
        ]
    elif confidence < 75:
        templates = [
            "{r} seems about right, give or take a universe.",
            "I'm not 100% sure, but {r} has good vibes.",
            "Let's go with {r} and walk away confidently."
        ]
    else:
        templates = [
            "{r}. Final answer. No questions.",
            "{r} is obviously correct and I refuse to elaborate.",
            "If {r} is wrong, then math is wrong."
        ]

    return random.choice(templates).format(r=result)


In [49]:
tests = [
    (100, 100, "+"),
    (100, 100, "/"),
    (35, 21, "*"),
    (10, 4, "-")
]

for a, b, op in tests:
    res, conf = goofy_calc(a, b, op)

    expr = f"{a} {op} {b}"   # <-- THIS WAS MISSING

    comment = goofy_comment(expr, res, conf)


    print(f"{expr} = {res}  (confidence: {conf:.1f}%)")
    print(comment)
    print()


100 + 100 = 198.0472  (confidence: 99.0%)
LLM says: 198.0472. Final answer. No questions.

100 / 100 = 2.9248  (confidence: 34.2%)
LLM says: Uhh… I think it's 2.9248, but please don't quote me.

35 * 21 = 245.3125  (confidence: 33.4%)
LLM says: Uhh… I think it's 245.3125, but please don't quote me.

10 - 4 = 6.0928  (confidence: 98.5%)
LLM says: 6.0928 is obviously correct and I refuse to elaborate.



In [51]:
while True:
  print("Do you really wanna do math?(Y/N)")
  inp = input(":")
  if inp.upper() =="Y":
    try:
        a1 = int(input("Enter the first number: "))
        a2 = int(input("Enter the second number: "))
    except ValueError:
        print("Numbers. I asked for numbers.\n")
        continue
    sign=input("Enter the operation (+,-,/,*):")
    if sign not in ["+","-","/","*"]:
      print("Invalid operation, now type it all from start >:)")
      continue
    res, conf = goofy_calc(a1, a2, sign)
    expr = f"{a1} {sign} {a2}"   # <-- THIS WAS MISSING

    comment = goofy_comment(expr, res, conf)

    print( comment)
    print()
  elif inp.upper()=="N":
    print("Great Choice! I hate math two!")#intentional
    print()
    break
  else:
    print("Stop entering invalid input weezo.")
    print()

#
#     Summary:
#     This project demonstrates why neural networks are poor tools
#     for exact symbolic computation. They learn patterns from data,
#     not rules, and their confidence does not imply correctness,
#     especially outside the training distribution.
#

Do you really wanna do math?(Y/N)
:n
Great Choice! I hate math two!

